So, you have:

1. WordCount.java  
    ```
    package me.nikhil;

    import org.apache.hadoop.conf.Configuration;
    import org.apache.hadoop.fs.Path;
    import org.apache.hadoop.io.IntWritable;
    import org.apache.hadoop.io.Text;
    import org.apache.hadoop.mapreduce.Job;
    import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
    import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

    import java.io.IOException;

    public class WordCount {

        /* Requires input text file and output path */
        public static void main(String args[]) throws Exception {

            if (args.length != 2) {
                System.err.println("Invalid Command");
                System.err.println("Usage: WordCount <input path> <output path>");
                System.exit(0);
            }

            Configuration conf = new Configuration();

            Job job = new Job(conf, "wordcount");
            job.setJarByClass(WordCount.class);
            FileInputFormat.addInputPath(job, new Path(args[0]));
            FileOutputFormat.setOutputPath(job, new Path(args[1]));
            job.setMapperClass(WordCountMapper.class);
            job.setReducerClass(WordCountReducer.class);
            job.setOutputKeyClass(Text.class);  // For Reducer, and implicitly for Mapper also
            job.setOutputValueClass(IntWritable.class);  // For Reducer, and implicitly for Mapper also

            System.exit(job.waitForCompletion(true) ? 0 : 1);

        }
    }
    ```

2. WordCountMapper.java  
    ```
    package me.nikhil;

    import org.apache.hadoop.io.IntWritable;
    import org.apache.hadoop.io.LongWritable;
    import org.apache.hadoop.io.Text;
    import org.apache.hadoop.mapreduce.Mapper;

    import java.io.IOException;

    public class WordCountMapper extends Mapper<LongWritable, Text, Text, IntWritable> {
        /* 
            Generic class with the following data types to specify
            <input key type, 
             input value type, 
             output key type, 
             output value type>
        */

        @Override
        public void map(LongWritable key, Text value, Context context) throws IOException, InterruptedException {
            String line = value.toString();
            for (String word : line.split(" ")) {
                if (word.length() > 0) {
                    context.write(new Text(word), new IntWritable(1));
                }
            }
        }
    }
    ```

3. WordCountReducer.java  
    ```
    package me.nikhil;

    import org.apache.hadoop.io.IntWritable;
    import org.apache.hadoop.io.Text;
    import org.apache.hadoop.mapreduce.Reducer;

    import java.io.IOException;

    public class WordCountReducer extends Reducer<Text, IntWritable, Text, IntWritable> {

        /* 
            Generic class with the following data types to specify
            <input key type, 
             input value type, 
             output key type, 
             output value type>
        */

        @Override
        public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
            int count = 0;
            for (IntWritable value : values) {
                count += value.get();
            }

            context.write(key, new IntWritable(count));
        }
    }
    ```

What do you do next?

**First, you create a jar**

```
$ ls -lrt
total 40
-rw-r--r--  1 genericuser  staff   752 Sep 26 12:40 WordCountReducer.java
-rw-r--r--  1 genericuser  staff  3233 Sep 26 12:57 input.txt
-rw-r--r--  1 genericuser  staff   835 Sep 26 13:52 WordCountMapper.java
-rw-r--r--  1 genericuser  staff  1341 Sep 26 13:53 WordCount.java
```

```
$ export CLASSPATH="/opt/hadoop-2.8.0/share/hadoop/mapreduce/hadoop-mapreduce-client-core-2.8.0.jar:/opt/hadoop-2.8.0/share/hadoop/mapreduce/hadoop-mapreduce-client-common-2.8.0.jar:/opt/hadoop-2.8.0/share/hadoop/common/hadoop-common-2.8.0.jar"
```

```
$ javac -d . WordCount.java WordCountReducer.java WordCountMapper.java
```

```
$ ls -R me
nikhil

me/nikhil:
WordCount.class        WordCountMapper.class  WordCountReducer.class
```

```
$ printf "Main-Class: me.nikhil.WordCount\n\n" > Manifest.txt
```

```
$ jar cfm wordcount.jar Manifest.txt me/nikhil/*.class
```

**Then make sure your input file is sent to hdfs**

```
$ hadoop fs -mkdir /test

$ hadoop fs -put input.txt /test

$ hadoop fs -ls /test
17/09/26 14:42:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   1 genericuser supergroup       3233 2017-09-26 13:00 /test/input.txt
```

**Now you can ran hadoop**

```
$ hadoop jar wordcount.jar /test /wordcountoutput

17/09/26 13:58:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/09/26 13:58:04 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/09/26 13:58:06 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 0 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:07 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 1 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:08 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 2 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:09 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 3 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:10 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 4 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:11 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 5 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:12 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 6 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:13 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 7 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:14 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 8 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:15 INFO ipc.Client: Retrying connect to server: 0.0.0.0/0.0.0.0:8032. Already tried 9 time(s); retry policy is RetryUpToMaximumCountWithFixedSleep(maxRetries=10, sleepTime=1000 MILLISECONDS)
17/09/26 13:58:15 WARN ipc.Client: Failed to connect to server: 0.0.0.0/0.0.0.0:8032: retries get failed due to exceeded maximum allowed retries number: 10
java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:206)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:531)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:495)
	at org.apache.hadoop.ipc.Client$Connection.setupConnection(Client.java:681)
	at org.apache.hadoop.ipc.Client$Connection.setupIOstreams(Client.java:777)
	at org.apache.hadoop.ipc.Client$Connection.access$3500(Client.java:409)
	at org.apache.hadoop.ipc.Client.getConnection(Client.java:1542)
	at org.apache.hadoop.ipc.Client.call(Client.java:1373)
	at org.apache.hadoop.ipc.Client.call(Client.java:1337)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:227)
	at org.apache.hadoop.ipc.ProtobufRpcEngine$Invoker.invoke(ProtobufRpcEngine.java:116)
	at com.sun.proxy.$Proxy13.getNewApplication(Unknown Source)
	at org.apache.hadoop.yarn.api.impl.pb.client.ApplicationClientProtocolPBClientImpl.getNewApplication(ApplicationClientProtocolPBClientImpl.java:258)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at org.apache.hadoop.io.retry.RetryInvocationHandler.invokeMethod(RetryInvocationHandler.java:398)
	at org.apache.hadoop.io.retry.RetryInvocationHandler$Call.invokeMethod(RetryInvocationHandler.java:163)
	at org.apache.hadoop.io.retry.RetryInvocationHandler$Call.invoke(RetryInvocationHandler.java:155)
	at org.apache.hadoop.io.retry.RetryInvocationHandler$Call.invokeOnce(RetryInvocationHandler.java:95)
	at org.apache.hadoop.io.retry.RetryInvocationHandler.invoke(RetryInvocationHandler.java:335)
	at com.sun.proxy.$Proxy14.getNewApplication(Unknown Source)
	at org.apache.hadoop.yarn.client.api.impl.YarnClientImpl.getNewApplication(YarnClientImpl.java:242)
	at org.apache.hadoop.yarn.client.api.impl.YarnClientImpl.createApplication(YarnClientImpl.java:250)
	at org.apache.hadoop.mapred.ResourceMgrDelegate.getNewJobID(ResourceMgrDelegate.java:193)
	at org.apache.hadoop.mapred.YARNRunner.getNewJobID(YARNRunner.java:241)
	at org.apache.hadoop.mapreduce.JobSubmitter.submitJobInternal(JobSubmitter.java:155)
	at org.apache.hadoop.mapreduce.Job$11.run(Job.java:1341)
	at org.apache.hadoop.mapreduce.Job$11.run(Job.java:1338)
	at java.security.AccessController.doPrivileged(Native Method)
	at javax.security.auth.Subject.doAs(Subject.java:422)
	at org.apache.hadoop.security.UserGroupInformation.doAs(UserGroupInformation.java:1807)
	at org.apache.hadoop.mapreduce.Job.submit(Job.java:1338)
	at org.apache.hadoop.mapreduce.Job.waitForCompletion(Job.java:1359)
	at me.nikhil.WordCount.main(WordCount.java:35)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at org.apache.hadoop.util.RunJar.run(RunJar.java:234)
	at org.apache.hadoop.util.RunJar.main(RunJar.java:148)
```

**But of course it doesn't run the first time.** Because you want to make sure you have the following change made:

```
$ vim $HADOOP_HOME/etc/hadoop/yarn-site.xml

<configuration>
<!-- Site specific YARN configuration properties -->
    <property>
        <name>yarn.nodemanager.aux-services</name>
        <value>mapreduce_shuffle</value>
    </property>
    <property>
        <name>yarn.resourcemanager.hostname</name>
        <value>127.0.0.1</value>
    </property>
</configuration>
```

Don't forget to stop and start yarn!

```
$ stop-yarn.sh
$ start-yarn.sh
```

**Does it run now? Of course not...**

```
$ hadoop jar wordcount.jar /test /wordcountoutput

17/09/26 14:49:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
17/09/26 14:49:13 INFO client.RMProxy: Connecting to ResourceManager at /127.0.0.1:8032
17/09/26 14:49:14 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
17/09/26 14:49:15 INFO input.FileInputFormat: Total input files to process : 1
17/09/26 14:49:15 INFO mapreduce.JobSubmitter: number of splits:1
17/09/26 14:49:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1506462487747_0001
17/09/26 14:49:16 INFO impl.YarnClientImpl: Submitted application application_1506462487747_0001
17/09/26 14:49:16 INFO mapreduce.Job: The url to track the job: http://generics-air:8088/proxy/application_1506462487747_0001/
17/09/26 14:49:16 INFO mapreduce.Job: Running job: job_1506462487747_0001
```

Gets stuck there forever...

**The configurations that finally worked are shown in ./config-files/**

In [1]:
ls -lrta config-files/

total 120
-rw-r--r--@  1 genericuser  staff   862 Sep 26 16:27 mapred-site.xml
-rw-r--r--   1 genericuser  staff   673 Sep 26 16:27 .bashrc
-rw-r--r--   1 genericuser  staff  3084 Sep 26 16:37 wordcount.jar
-rwxr-xr-x   1 genericuser  staff    65 Sep 26 16:51 run-test2.sh*
-rwxr-xr-x   1 genericuser  staff    48 Sep 26 16:55 copy-new-config.sh*
-rwxr-xr-x   1 genericuser  staff   303 Sep 26 17:01 run-test1.sh*
-rw-r--r--@  1 genericuser  staff   926 Sep 26 17:05 yarn-site.xml
-rw-r--r--@  1 genericuser  staff  1391 Sep 26 17:25 hdfs-site.xml
-rw-r--r--@  1 genericuser  staff   880 Sep 26 17:35 core-site.xml
-rw-r--r--   1 genericuser  staff   411 Sep 26 17:40 hosts
-rw-r--r--@  1 genericuser  staff  4718 Sep 26 17:55 hadoop-env.sh
-rw-r--r--   1 genericuser  staff  6800 Sep 26 17:57 test1
-rw-r--r--   1 genericuser  staff  3086 Sep 26 17:58 test2
drwxr-xr-x  16 genericuser  staff   544 Sep 26 18:13 ./
drwxr-xr-x  12 genericuser  staff   408 Sep 26 18:14 .git/
drwxr-xr-x   6 genericuser